In [16]:
import pandas as pd
import numpy as np
import requests
from yahooquery import Ticker  # Step 2 (Speed)
import yfinance as yf          # Step 3 (Reliability)
import io
import json
import os
import time


In [17]:
import os
import json

# ==========================================
# CONFIGURATION
# ==========================================
# 1. FOLDER SETUP (The Fix for GitHub Portability)
DATA_FOLDER = "YfinanceDataDump"  # Relative path (creates folder in project root)

# Create the folder if it doesn't exist
if not os.path.exists(DATA_FOLDER):
    try:
        os.makedirs(DATA_FOLDER)
        print(f"Created data folder: {DATA_FOLDER}")
    except Exception as e:
        print(f"Warning: Could not create folder '{DATA_FOLDER}'. Saving to current directory. Error: {e}")
        DATA_FOLDER = "."

# 2. FILE PATHS (Everything saves inside the folder now)
CACHE_FILE = os.path.join(DATA_FOLDER, "financial_cache.json")
FORTRESS_CSV = os.path.join(DATA_FOLDER, "fortress_stocks.csv")
STRONG_CSV = os.path.join(DATA_FOLDER, "strong_stocks.csv")
RISKY_CSV = os.path.join(DATA_FOLDER, "risky_stocks.csv")
ANALYST_CSV = os.path.join(DATA_FOLDER, "Analyst_Fortress_Picks.csv")
BUFFETT_CSV = os.path.join(DATA_FOLDER, "Buffett_Value_Picks.csv")

# 3. UNIVERSE FILTERS
MIN_PRICE = 2.00               
MIN_VOLUME = 1_000_000       
MIN_CAP = 300_000_000        # $300M
MIN_CURRENT_RATIO = 1.2
MAX_PE_RATIO = 100.0         

# 4. SAFETY THRESHOLDS 
MIN_INTEREST_COVERAGE = 1.5
MIN_ROIC = 0.05              # 5%
FORTRESS_MARGIN_THRESHOLD = 0.05  # 5%

EXCLUDED_SECTORS = ['Financial Services', 'Real Estate']
CACHE_EXPIRY_DAYS = 30 

# ==========================================
# HELPER FUNCTIONS
# ==========================================
def load_cache():
    if os.path.exists(CACHE_FILE):
        try:
            with open(CACHE_FILE, 'r') as f:
                return json.load(f)
        except:
            return {}
    return {}

def save_cache(cache_data):
    try:
        with open(CACHE_FILE, 'w') as f:
            json.dump(cache_data, f)
    except Exception as e:
        print(f"Warning: Could not save cache: {e}")

def calculate_altman_z_yfinance(bs, fin, market_cap):
    """
    Calculates Z-Score using yfinance DataFrames.
    Formula: 1.2A + 1.4B + 3.3C + 0.6D + 1.0E
    """
    try:
        # Helper to safely get value from Series
        def get_val(df, keys):
            for k in keys:
                if k in df.index:
                    return df.loc[k].iloc[0]
            return 0

        # Map yfinance row names
        total_assets = get_val(bs, ['Total Assets'])
        total_liab = get_val(bs, ['Total Liabilities Net Minority Interest', 'Total Liabilities'])
        current_assets = get_val(bs, ['Current Assets', 'Total Current Assets'])
        current_liab = get_val(bs, ['Current Liabilities', 'Total Current Liabilities'])
        retained_earnings = get_val(bs, ['Retained Earnings'])
        
        ebit = get_val(fin, ['EBIT', 'Operating Income'])
        total_revenue = get_val(fin, ['Total Revenue'])
        
        if total_assets == 0 or total_liab == 0: return 0

        # A: Working Capital / Total Assets
        A = (current_assets - current_liab) / total_assets
        
        # B: Retained Earnings / Total Assets
        B = retained_earnings / total_assets
        
        # C: EBIT / Total Assets
        C = ebit / total_assets
        
        # D: Market Value of Equity / Total Liabilities
        D = market_cap / total_liab
        
        # E: Sales / Total Assets
        E = total_revenue / total_assets
        
        return (1.2 * A) + (1.4 * B) + (3.3 * C) + (0.6 * D) + (1.0 * E)
    except Exception as e:
        return 0

In [18]:
# ==========================================
# STEP 1: FETCH COMBINED UNIVERSE (USA + CAD)
# ==========================================
def get_combined_universe():
    print("--- STEP 1: Fetching North American Universe ---")
    tickers = []
    
    # 1. USA
    try:
        url_us = "https://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        s = requests.get(url_us).content
        df_us = pd.read_csv(io.StringIO(s.decode('utf-8')), sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = [x.replace('$', '-') for x in df_us['Symbol'].astype(str) if len(x) < 5]
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US stocks.")
    except:
        print("   -> Error fetching USA list.")

    return tickers

In [19]:
# ==========================================
# STEP 2: LIGHTWEIGHT SIEVE (YahooQuery)
# ==========================================
def get_initial_survivors(tickers):
    print(f"\n--- STEP 2: Running 'Lightweight' Filter on {len(tickers)} stocks ---")
    chunk_size = 500 
    survivors = []
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        if i % 2 == 0: print(f" -> Processing Batch {i+1}/{len(chunks)}...")
        try:
            yq = Ticker(chunk, asynchronous=True)
            df_modules = yq.get_modules('summaryProfile summaryDetail financialData price defaultKeyStatistics')
            
            for symbol, data in df_modules.items():
                if isinstance(data, str): continue 
                try:
                    price = data.get('price', {}).get('regularMarketPrice', 0)
                    if price is None: price = 0
                    
                    vol = data.get('summaryDetail', {}).get('averageVolume', 0)
                    if vol is None or vol == 0:
                         vol = data.get('price', {}).get('averageDailyVolume10Day', 0)
                    
                    cap = data.get('price', {}).get('marketCap', 0)
                    if cap is None: cap = 0
                    
                    sector = data.get('summaryProfile', {}).get('sector', 'Unknown')
                    fin_data = data.get('financialData', {})
                    curr_ratio = fin_data.get('currentRatio', 0)
                    op_margins = fin_data.get('operatingMargins', 0)
                    if curr_ratio is None: curr_ratio = 0
                    if op_margins is None: op_margins = 0

                    # --- P/E RATIO CHECK ---
                    pe = data.get('summaryDetail', {}).get('trailingPE')
                    if pe is not None and pe > MAX_PE_RATIO: continue

                    # FILTERS
                    if price < MIN_PRICE: continue
                    if cap < MIN_CAP: continue
                    if vol < MIN_VOLUME: continue 
                    if any(x in sector for x in EXCLUDED_SECTORS): continue
                    if curr_ratio < MIN_CURRENT_RATIO: continue
                    if op_margins <= 0: continue 
                    
                    survivors.append({
                        'Ticker': symbol,
                        'Sector': sector,
                        'Price': price,
                        'Op Margin %': round(op_margins * 100, 2),
                        'P/E': round(pe, 2) if pe else 0,
                        'Curr Ratio': curr_ratio,
                        'Mkt Cap (B)': round(cap / 1_000_000_000, 2)
                    })
                except: continue
        except: continue
    return pd.DataFrame(survivors)

In [ ]:
# ==========================================
# STEP 3: DEEP DIVE (yfinance + Cache)
# ==========================================
def get_advanced_metrics(survivor_df):
    tickers = survivor_df['Ticker'].tolist()
    print(f"\n--- STEP 3: Fetching Deep Financials for {len(tickers)} Survivors ---")
    
    cache = load_cache()
    current_time = time.time()
    expiry_seconds = CACHE_EXPIRY_DAYS * 86400
    
    final_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 20 == 0: print(f" -> Analyzing {i+1}/{len(tickers)}: {ticker}...")
        
        # --- CRITICAL FIX: Anti-Throttle Sleep ---
        time.sleep(0.75)  # Sleep to avoid hitting Yahoo too fast change back to 1 second if throttling occurs
        # -----------------------------------------
        
        # Helper: Logic to assign Tier based on Average Margin & Safety
        def determine_tier_history(metrics, is_fortress_margin, is_pos_margin):
            # 1. Safety Checks (Must pass these regardless of margins)
            if metrics['int_cov'] < MIN_INTEREST_COVERAGE: return "Risky"
            if metrics['roic'] < MIN_ROIC: return "Risky"
            
            # 2. Historical Margin Checks (Using the 4-Year Average)
            if is_fortress_margin: 
                return "Fortress"  # Avg Margin > 5%
            elif is_pos_margin:
                return "Strong"    # Avg Margin > 0%
            
            return "Risky"         # Avg Margin was negative

        # 1. CHECK CACHE
        cached_data = cache.get(ticker)
        if cached_data and (current_time - cached_data['timestamp'] < expiry_seconds):
            if cached_data.get('roic') == -999: continue 
            # If using cache, we might miss the 'avg_margin' recalculation unless we force update.
            # ideally we proceed to fetch if we suspect cache is old logic, but for now we trust cache.
            # To force new logic, clear your cache file (delete financial_cache.json).
            pass 

        # 2. FETCH NEW DATA
        try:
            stock = yf.Ticker(ticker)
            fin = stock.financials
            bs = stock.balance_sheet
            
            # Check if Yahoo actually gave us data
            if fin.empty or bs.empty:
                # Don't cache this as a failure immediately; it might be a connection blip.
                # But to keep logic simple, we skip.
                print(f"   ⚠️ No data for {ticker} (skipping)")
                continue
            
            # --- A. NEW LOGIC: 4-Year Average Margin Check ---
            try:
                # Get Operating Income (try 'Operating Income' first, then 'EBIT')
                if 'Operating Income' in fin.index:
                    op_income_history = fin.loc['Operating Income']
                elif 'EBIT' in fin.index:
                    op_income_history = fin.loc['EBIT']
                else:
                    op_income_history = pd.Series([0]) 

                # Get Revenue
                revenue_history = fin.loc['Total Revenue']
                
                # Calculate Margins for every available year
                # This automatically handles 1, 2, 3, or 4 years of data
                yearly_margins = (op_income_history / revenue_history).dropna()
                
                if len(yearly_margins) > 0:
                    avg_margin = yearly_margins.mean()
                    
                    # The Uniform Rule: Is the AVERAGE above the threshold?
                    is_fortress_margin = avg_margin > FORTRESS_MARGIN_THRESHOLD
                    is_positive_margin = avg_margin > 0
                else:
                    is_fortress_margin = False
                    is_positive_margin = False

            except Exception as e:
                # Fail safe
                is_fortress_margin = False
                is_positive_margin = False
            # ---------------------------------------------------

            # --- B. Standard Calculations (Safety Checks) ---
            def get_item(df, keys):
                for k in keys:
                    if k in df.index: return df.loc[k].iloc[0]
                return 0

            ebit = get_item(fin, ['EBIT', 'Operating Income', 'Pretax Income'])
            int_exp = get_item(fin, ['Interest Expense', 'Interest Expense Non Operating'])
            total_assets = get_item(bs, ['Total Assets'])
            curr_liab = get_item(bs, ['Current Liabilities', 'Total Current Liabilities'])
            
            # Interest Coverage
            int_exp = abs(int_exp)
            if int_exp == 0: int_cov = 100
            else: int_cov = ebit / int_exp
            
            # ROIC
            invested_cap = total_assets - curr_liab
            if invested_cap <= 0: roic = 0
            else: roic = ebit / invested_cap
            
            # Z-Score
            base_row = survivor_df[survivor_df['Ticker'] == ticker].iloc[0]
            mkt_cap_raw = base_row['Mkt Cap (B)'] * 1_000_000_000
            z = calculate_altman_z_yfinance(bs, fin, mkt_cap_raw)
            
            # Cache the metrics
            metrics = {
                'timestamp': current_time,
                'z_score': round(z, 2),
                'roic': roic,
                'int_cov': round(int_cov, 2)
            }
            cache[ticker] = metrics
            
            # --- C. Determine Final Tier ---
            tier = determine_tier_history(metrics, is_fortress_margin, is_positive_margin)

            final_data.append({
                'Ticker': ticker,
                'Tier': tier,
                'Price': base_row['Price'],
                'P/E': base_row['P/E'],
                'Sector': base_row['Sector'],
                'Z-Score': round(z, 2),
                'ROIC %': round(roic * 100, 2),
                'Op Margin %': base_row['Op Margin %'], 
                'Avg Margin (4Y)': round(avg_margin * 100, 2) if 'avg_margin' in locals() else 0,
                'Curr Ratio': base_row['Curr Ratio'],
                'Int Cov': round(int_cov, 2),
                'Mkt Cap (B)': base_row['Mkt Cap (B)']
            })
            
        except Exception as e:
            continue

    save_cache(cache)
    return pd.DataFrame(final_data)

In [21]:
# ==========================================
# MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    tickers = get_combined_universe()
    
    if len(tickers) > 0:
        survivors_df = get_initial_survivors(tickers)
        
        if not survivors_df.empty:
            print(f"\n✅ Step 2 Complete. {len(survivors_df)} stocks passed basic filters.")
            
            final_results = get_advanced_metrics(survivors_df)
            
            if not final_results.empty:
                final_results = final_results.sort_values(by=['Tier', 'Z-Score'], ascending=[True, False])
                
                # 1. Standard Split
                fortress_df = final_results[final_results['Tier'] == 'Fortress'].copy()
                strong_df = final_results[final_results['Tier'] == 'Strong'].copy()
                risky_df = final_results[final_results['Tier'] == 'Risky'].copy()
                
                # 3. Save Files (Updated to use Relative Paths from Cell 2)
                try:
                    fortress_df.to_csv(FORTRESS_CSV, index=False)
                    strong_df.to_csv(STRONG_CSV, index=False)
                    risky_df.to_csv(RISKY_CSV, index=False)
                    
                    print("\n" + "="*60)
                    print("RESULTS GENERATED")
                    print("="*60)
                    print(f"1. FORTRESS ({len(fortress_df)}): Saved to '{FORTRESS_CSV}'")
                    print(f"2. STRONG   ({len(strong_df)}): Saved to '{STRONG_CSV}'")
                    print(f"3. RISKY    ({len(risky_df)}): Saved to '{RISKY_CSV}'")
                except Exception as e:
                    print(f"\n⚠️ Error Saving Files: {e}")
                    print("Check if the file is open in Excel or if the folder exists.")
                
                pd.set_option('display.max_rows', 500)
                pd.set_option('display.max_columns', 20)
                pd.set_option('display.width', 1000)
                
                print("\n--- FORTRESS PREVIEW ---")
                print(fortress_df.head(15))
            else:
                print("No stocks passed the deep financial analysis.")
        else:
            print("No stocks passed the initial lightweight filter.")
    else:
        print("Could not fetch ticker universe.")

--- STEP 1: Fetching North American Universe ---
   -> Found 6013 US stocks.

--- STEP 2: Running 'Lightweight' Filter on 6013 stocks ---
 -> Processing Batch 1/13...
 -> Processing Batch 3/13...
 -> Processing Batch 5/13...
 -> Processing Batch 7/13...
 -> Processing Batch 9/13...
 -> Processing Batch 11/13...
 -> Processing Batch 13/13...

✅ Step 2 Complete. 448 stocks passed basic filters.

--- STEP 3: Fetching Deep Financials for 448 Survivors ---
 -> Analyzing 1/448: A...
 -> Analyzing 21/448: ALGN...
 -> Analyzing 41/448: AQN...
 -> Analyzing 61/448: AXL...
 -> Analyzing 81/448: BMBL...
 -> Analyzing 101/448: CDE...
 -> Analyzing 121/448: COP...
 -> Analyzing 141/448: CXM...
 -> Analyzing 161/448: DRS...
 -> Analyzing 181/448: ET...
 -> Analyzing 201/448: FND...
 -> Analyzing 221/448: GILD...
 -> Analyzing 241/448: HAL...
 -> Analyzing 261/448: HUN...
 -> Analyzing 281/448: ITGR...
 -> Analyzing 301/448: LFST...
 -> Analyzing 321/448: MGY...
 -> Analyzing 341/448: NEM...
 -> Anal

In [22]:
# 1. Define the function (if you haven't already in a previous cell)
def get_analyst_fortress_from_var(df_input):
    working_df = df_input.copy()
    tickers = working_df['Ticker'].tolist()
    
    print(f"\n--- STEP 4: Fetching Analyst Ratings for {len(tickers)} Stocks (From Memory) ---")
    print("    (Fetching serially to avoid throttling...)")
    
    analyst_data = []
    
    for i, ticker in enumerate(tickers):
        if i % 10 == 0: print(f" -> Analyst Scan {i+1}/{len(tickers)}: {ticker}...")
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            rec_mean = info.get('recommendationMean')
            target_price = info.get('targetMeanPrice')
            current_price = info.get('currentPrice')
            
            # Filter: Must be better than 2.0 (Lower is better)
            if rec_mean is None or rec_mean > 2.0: continue
            
            upside = 0
            if target_price and current_price:
                upside = round(((target_price - current_price) / current_price) * 100, 2)
            
            # Merge with existing data
            base_row = working_df[working_df['Ticker'] == ticker].iloc[0].to_dict()
            base_row['Analyst_Rating'] = rec_mean
            base_row['Target_Price'] = target_price
            base_row['Upside_%'] = upside
            
            analyst_data.append(base_row)
            time.sleep(0.2) # Polite delay
            
        except Exception:
            continue
            
    return pd.DataFrame(analyst_data)

# ==========================================
# 2. EXECUTE IT (Run this part!)
# ==========================================

# Check if fortress_df exists from the previous step
if 'fortress_df' in locals() and not fortress_df.empty:
    
    # Run the function
    Analyst_Fortress_DF = get_analyst_fortress_from_var(fortress_df)
    
    if not Analyst_Fortress_DF.empty:
        # Sort by best Analyst Rating (Lower is better) or Upside
        Analyst_Fortress_DF = Analyst_Fortress_DF.sort_values(by='Upside_%', ascending=False)
        
        # Display Results
        print("\n✅ Analyst Scan Complete!")
        print(f"Found {len(Analyst_Fortress_DF)} stocks with Buy Ratings (Score < 2.0)")
        
        # Save to CSV
        Analyst_Fortress_DF.to_csv(ANALYST_CSV, index=False)
        print("Saved to 'Analyst_Fortress_Picks.csv'")
        
        # Show top picks
        cols = ['Ticker', 'Price', 'Analyst_Rating', 'Upside_%', 'Target_Price', 'Tier']
        print(Analyst_Fortress_DF[cols].head(20))
    else:
        print("No stocks passed the Analyst filter.")
else:
    print("❌ 'fortress_df' not found or empty. Please run the Main Filter (Step 1-3) first.")


--- STEP 4: Fetching Analyst Ratings for 267 Stocks (From Memory) ---
    (Fetching serially to avoid throttling...)
 -> Analyst Scan 1/267: NVDA...
 -> Analyst Scan 11/267: ASM...
 -> Analyst Scan 21/267: EW...
 -> Analyst Scan 31/267: LLY...
 -> Analyst Scan 41/267: CGNX...
 -> Analyst Scan 51/267: SHOO...
 -> Analyst Scan 61/267: QCOM...
 -> Analyst Scan 71/267: BRBR...
 -> Analyst Scan 81/267: BSX...
 -> Analyst Scan 91/267: ECL...
 -> Analyst Scan 101/267: ON...
 -> Analyst Scan 111/267: SMCI...
 -> Analyst Scan 121/267: AEO...
 -> Analyst Scan 131/267: PPG...
 -> Analyst Scan 141/267: B...
 -> Analyst Scan 151/267: IR...
 -> Analyst Scan 161/267: BURL...
 -> Analyst Scan 171/267: BIIB...
 -> Analyst Scan 181/267: COP...
 -> Analyst Scan 191/267: EQNR...
 -> Analyst Scan 201/267: M...
 -> Analyst Scan 211/267: EGO...
 -> Analyst Scan 221/267: OC...
 -> Analyst Scan 231/267: AM...
 -> Analyst Scan 241/267: JAZZ...
 -> Analyst Scan 251/267: BIDU...
 -> Analyst Scan 261/267: PAGS...

In [23]:
import pandas as pd
from yahooquery import Ticker

# ==========================================
# BUFFETT "BELOW NAV" SCAN
# ==========================================
def get_buffett_value_picks(df_input):
    print(f"\n--- STEP 5: Warren Buffett 'Below NAV' Scan ---")
    print(f"    Scanning {len(df_input)} candidates for Deep Value...")
    print("    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%")

    tickers = df_input['Ticker'].tolist()
    buffett_candidates = []

    # Use YahooQuery for speed (Key Stats are summary data, no throttling risk here)
    chunk_size = 250
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]

    for chunk in chunks:
        try:
            yq = Ticker(chunk, asynchronous=True)
            # We need defaultKeyStatistics (P/B) and financialData (ROE, Debt)
            data = yq.get_modules("defaultKeyStatistics financialData")

            for symbol in chunk:
                if isinstance(data, dict) and symbol in data:
                    try:
                        stats = data[symbol].get('defaultKeyStatistics', {})
                        fin = data[symbol].get('financialData', {})

                        # 1. Price to Book < 1.0 (The Core Rule)
                        pb = stats.get('priceToBook')
                        # Skip if None, > 1.0, or negative (insolvent)
                        if pb is None or pb >= 1.0 or pb <= 0: continue

                        # 2. Positive ROE (No Zombies)
                        roe = fin.get('returnOnEquity', 0)
                        if roe is None or roe <= 0: continue

                        # 3. Reasonable Debt (Safety)
                        # Buffett hates high leverage on weak companies
                        de = fin.get('debtToEquity', 0)
                        if de is None or de > 100: continue 

                        # Get base data from input_df
                        base_row = df_input[df_input['Ticker'] == symbol].iloc[0].to_dict()

                        # Add new Value Metrics
                        base_row['P/B Ratio'] = round(pb, 2)
                        base_row['ROE %'] = round(roe * 100, 2)
                        base_row['Debt/Eq %'] = round(de, 2)

                        buffett_candidates.append(base_row)

                    except: continue
        except: continue

    return pd.DataFrame(buffett_candidates)

# ==========================================
# EXECUTION BLOCK
# ==========================================
# Ensure we have the 'final_results' from the Main Filter
if 'final_results' in locals() and not final_results.empty:
    
    Buffett_Value_DF = get_buffett_value_picks(final_results)
    
    if not Buffett_Value_DF.empty:
        # Sort by P/B Ratio (Cheapest first)
        Buffett_Value_DF = Buffett_Value_DF.sort_values(by='P/B Ratio', ascending=True)
        
        # Save results
        Buffett_Value_DF.to_csv(BUFFETT_CSV, index=False)
        
        print("\n" + "="*60)
        print("BUFFETT SCAN COMPLETE")
        print("="*60)
        print(f"Found {len(Buffett_Value_DF)} Deep Value Stocks (Trading < Book Value)")
        print("Saved to: 'Buffett_Value_Picks.csv'")
        
        # Display
        pd.set_option('display.max_rows', 500)
        cols = ['Ticker', 'Price', 'P/B Ratio', 'ROE %', 'Debt/Eq %', 'Sector', 'Tier']
        print("\n--- DEEP VALUE PICKS ---")
        print(Buffett_Value_DF[cols].head(20))
        
    else:
        print("\n❌ No stocks passed the Buffett Value filter (All stocks are trading > Book Value).")
else:
    print("❌ 'final_results' variable not found. Please run the Main Filter first.")


--- STEP 5: Warren Buffett 'Below NAV' Scan ---
    Scanning 444 candidates for Deep Value...
    Criteria: P/B < 1.0 (Below Book) | ROE > 0% (Profitable) | Debt/Eq < 100%

BUFFETT SCAN COMPLETE
Found 15 Deep Value Stocks (Trading < Book Value)
Saved to: 'Buffett_Value_Picks.csv'

--- DEEP VALUE PICKS ---
   Ticker   Price  P/B Ratio  ROE %  Debt/Eq %                  Sector      Tier
0    GMAB   30.80       0.33  29.41       2.47              Healthcare  Fortress
11     KT   18.97       0.38   5.08      60.17  Communication Services     Risky
7    ACHC   14.19       0.41   3.69      74.49              Healthcare  Fortress
10   ANGI   12.93       0.57   3.42      54.03  Communication Services     Risky
14    HLX    6.27       0.59   2.71      39.52                  Energy    Strong
3    MOMO    6.55       0.64   6.72       1.43  Communication Services  Fortress
2      MT   45.57       0.64   4.72      26.23         Basic Materials  Fortress
4     GGB    3.69       0.74   5.51      36.

In [24]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown
import warnings

# Ignore these specific FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

# ==========================================
# INSIDER FILTER FUNCTION (With Price)
# ==========================================
def filter_for_insider_buying(tickers):
    print(f"🕵️ Scanning {len(tickers)} stocks for Insider Buying & Price...")
    insider_picks = []
    
    # Chunk to prevent timeouts
    chunk_size = 20
    chunks = [tickers[i:i + chunk_size] for i in range(0, len(tickers), chunk_size)]
    
    for chunk in chunks:
        try:
            # Initialize Ticker object for the chunk
            yq = Ticker(chunk, asynchronous=True)
            
            # 1. Fetch Insider Transactions
            df_insiders = yq.insider_transactions
            
            # 2. Fetch Price Data (New Step)
            # This returns a dictionary: {'TICKER': {'regularMarketPrice': 10.50, ...}}
            price_data = yq.price
            
            # Validation: Ensure we have data to work with
            if isinstance(df_insiders, dict) or not hasattr(df_insiders, 'reset_index'): 
                continue
            
            df_insiders = df_insiders.reset_index()

            for symbol in chunk:
                if symbol not in df_insiders['symbol'].values:
                    continue
                
                # --- INSIDER LOGIC ---
                stock_tx = df_insiders[df_insiders['symbol'] == symbol].copy()
                
                buys = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Purchase", case=False, na=False)]
                sells = stock_tx[stock_tx['transactionText'].astype(str).str.contains("Sale", case=False, na=False)]
                
                buy_vol = buys['shares'].sum() if not buys.empty else 0
                sell_vol = sells['shares'].sum() if not sells.empty else 0
                
                # --- PRICE LOGIC ---
                current_price = None
                try:
                    # Safely attempt to grab the price from the dictionary
                    if isinstance(price_data, dict) and symbol in price_data:
                        current_price = price_data[symbol].get('regularMarketPrice', None)
                except Exception:
                    current_price = None

                # Only keep if Net Buying is Positive
                if buy_vol > sell_vol:
                    insider_picks.append({
                        'Ticker': symbol,
                        'Current_Price': current_price, # <--- New Column
                        'Insider_Buys_Count': len(buys),
                        'Net_Shares_Bought': buy_vol - sell_vol
                    })
                        
        except Exception as e:
            # print(f"Error on chunk: {e}") # Uncomment for debugging
            continue

    return pd.DataFrame(insider_picks)

# ==========================================
# 2. CREATE 'Fortress_insiders' DATAFRAME
# ==========================================

# Use fortress_df if it exists, otherwise use the top 20 backup list
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    print("⚠️ 'fortress_df' not found or empty.")

# Run the filter
Fortress_insiders = filter_for_insider_buying(target_tickers)

# Display so Data Wrangler picks it up
print(f"✅ Created 'Fortress_insiders' with {len(Fortress_insiders)} rows.")
display(Fortress_insiders)

🕵️ Scanning 267 stocks for Insider Buying & Price...
✅ Created 'Fortress_insiders' with 25 rows.


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought
0,NFLX,93.76,3,68670.0
1,KEYS,203.19,15,1368070.0
2,GIL,62.46,110,1575676.0
3,HXL,73.90,6,14327.0
4,SHLS,8.50,3,8335.0
5,CGAU,14.37,91,5359745.0
6,DKS,197.97,1,2000.0
7,AMKR,39.48,10,3529615.0
8,FSM,9.81,4,916900.0
9,OPCH,31.86,8,18954.0


In [25]:

# ==========================================
# 2. ANALYST FILTER FUNCTION FOR INSIDER PICKS (NEW)
# ==========================================
def filter_for_analyst_ratings(df_insiders, max_score=2.5):
    """
    Fetches analyst data for the insider winners and filters for 'Buy' or better.
    Scale: 1.0 = Strong Buy, 5.0 = Sell.
    Cutoff: 2.5 ensures we get 'Buy' and 'Strong Buy'.
    """
    if df_insiders.empty:
        return df_insiders
        
    tickers = df_insiders['Ticker'].tolist()
    
    
    try:
        yq = Ticker(tickers, asynchronous=True)
        # 'financial_data' contains the specific recommendation scores
        fin_data = yq.financial_data
        
        analyst_data = []
        for t in tickers:
            # Check if we got valid data for this ticker
            if isinstance(fin_data, dict) and t in fin_data:
                data = fin_data[t]
                # Ensure it's a dictionary and has the key we need
                if isinstance(data, dict) and 'recommendationMean' in data:
                    score = data.get('recommendationMean')
                    
                    # Only keep valid scores (sometimes they are None)
                    if score is not None:
                        analyst_data.append({
                            'Ticker': t,
                            'Analyst_Score': score,
                            'Analyst_Verdict': data.get('recommendationKey', 'N/A')
                        })
        
        df_analyst = pd.DataFrame(analyst_data)
        
        if df_analyst.empty:
            print("⚠️ No Analyst ratings found for these tickers.")
            return df_insiders # Return original if no data found
            
        # Merge with the Insider DataFrame
        merged = pd.merge(df_insiders, df_analyst, on='Ticker', how='inner')
        
        # FILTER: Keep only scores <= max_score (Lower is better)
        final_df = merged[merged['Analyst_Score'] <= max_score].copy()
        
        print(f"✅ Analyst Filter: {len(merged)} -> {len(final_df)} stocks (Min Rating: Buy).")
        return final_df.sort_values(by='Analyst_Score', ascending=True)

    except Exception as e:
        print(f"❌ Error in Analyst Filter: {e}")
        return df_insiders

# ==========================================
# 3. EXECUTION PIPELINE
# ==========================================

# A. Setup Tickers
if 'fortress_df' in locals() and not fortress_df.empty:
    target_tickers = fortress_df['Ticker'].tolist()
else:
    # Backup list just in case
    target_tickers = [
        'PET.TO', 'MFI.TO', 'TXG.TO', 'SAP.TO', 'PAAS.TO', 'NEO.TO', 'WPM.TO', 
        'FNV.TO', 'LUG.TO', 'DPM.TO', 'ASM.TO', 'PNG.V', 'DSG.TO', 'KNT.TO', 
        'GGD.TO', 'GRGD.TO', 'WDO.TO', 'OGC.TO', 'DNG.TO', 'CLS.TO'
    ]

# B. Run Insider Filter
insider_winners = filter_for_insider_buying(target_tickers)

# C. Run Analyst Filter (NEW STEP)
# We overwrite 'Fortress_insiders' so it works with your Data Wrangler flow
if not insider_winners.empty:
    Fortress_insiders_Analyst_buy = filter_for_analyst_ratings(insider_winners, max_score=2.5)
else:
    Fortress_insiders_Analyst_buy = pd.DataFrame()

# D. Display Result
if not Fortress_insiders_Analyst_buy.empty:
    print(f"\n🚀 Final List: {len(Fortress_insiders_Analyst_buy)} stocks (Fortress + Insider Buying + Analyst Buy Rating)")
    display(Fortress_insiders_Analyst_buy)
else:
    print("No stocks passed all filters.")

🕵️ Scanning 267 stocks for Insider Buying & Price...
✅ Analyst Filter: 15 -> 13 stocks (Min Rating: Buy).

🚀 Final List: 13 stocks (Fortress + Insider Buying + Analyst Buy Rating)


,Ticker,Current_Price,Insider_Buys_Count,Net_Shares_Bought,Analyst_Score,Analyst_Verdict
10,DAR,36.00,1,1120.0,1.38462,strong_buy
5,NXPI,217.06,3,413316.0,1.53125,buy
14,SBLK,19.22,8,143034.0,1.75000,buy
7,NWSA,26.12,73,53965480.0,1.75000,buy
1,KEYS,203.19,15,1368070.0,1.76923,buy
0,NFLX,93.76,3,68670.0,1.95349,buy
6,BRKR,47.11,4,41837.0,2.06667,buy
3,DKS,197.97,1,2000.0,2.07407,buy
9,DEO,86.27,11,703246.0,2.37500,buy
12,AMGN,327.31,11,72178.0,2.40625,buy


In [31]:
import pandas as pd
from yahooquery import Ticker
from IPython.display import display, Markdown

# ==========================================
# BURRY "RELATIVE VALUE" FILTER (EV/EBITDA)
# ==========================================
def filter_burry_ev_ebitda(df_input):
    if df_input is None or df_input.empty:
        print("❌ Input DataFrame is empty.")
        return pd.DataFrame()

    print(f"📉 Analyzing EV/EBITDA for {len(df_input)} Fortress stocks...")
    
    tickers = df_input['Ticker'].tolist()
    
    # 1. BATCH FETCH DATA (YahooQuery for Speed)
    # We fetch multiple modules now to ensure we have ingredients for manual calc
    try:
        yq = Ticker(tickers, asynchronous=True)
        # Fetch key stats (pre-calced), financial data (debt/cash), and summary (market cap)
        data = yq.get_modules('defaultKeyStatistics financialData summaryDetail')
    except Exception as e:
        print(f"❌ Error fetching data: {e}")
        return pd.DataFrame()

    ev_data = []
    
    # 2. PARSE DATA
    for ticker in tickers:
        try:
            # Safe extraction of modules
            ticker_data = data.get(ticker, {})
            if isinstance(ticker_data, str): continue # Handle API errors
            
            stats = ticker_data.get('defaultKeyStatistics', {})
            fin_data = ticker_data.get('financialData', {})
            summary = ticker_data.get('summaryDetail', {})

            # --- PLAN A: Pre-calculated Metric ---
            ev_ebitda = stats.get('enterpriseToEbitda')
            
            # --- PLAN B: Manual Calculation (The Fallback) ---
            if ev_ebitda is None:
                try:
                    # We need all 4 components to calculate it manually
                    market_cap = summary.get('marketCap')
                    total_debt = fin_data.get('totalDebt')
                    total_cash = fin_data.get('totalCash')
                    ebitda = fin_data.get('ebitda')
                    
                    if all(v is not None for v in [market_cap, total_debt, total_cash, ebitda]):
                        if ebitda != 0:
                            # Formula: EV = Market Cap + Debt - Cash
                            enterprise_value = market_cap + total_debt - total_cash
                            ev_ebitda = enterprise_value / ebitda
                            # print(f"   -> Manual Calc success for {ticker}: {round(ev_ebitda, 2)}")
                except Exception:
                    pass # If manual calc fails, we just skip

            # Filter: We only want profitable EBITDA for valuation (exclude negatives/None)
            if ev_ebitda is not None and ev_ebitda > 0:
                ev_data.append({
                    'Ticker': ticker,
                    'EV/EBITDA': round(ev_ebitda, 2)
                })
        except:
            continue
            
    df_vals = pd.DataFrame(ev_data)
    
    if df_vals.empty:
        print("⚠️ Could not retrieve EV/EBITDA data (even with manual fallback).")
        return pd.DataFrame()

    # 3. MERGE WITH SECTOR DATA
    # We merge back with original DF to get the 'Sector' column
    merged_df = pd.merge(df_input, df_vals, on='Ticker', how='inner')
    
    # 4. CALCULATE SECTOR AVERAGES
    print("\n--- 📊 SECTOR AVERAGES (EV/EBITDA) ---")
    sector_stats = merged_df.groupby('Sector')['EV/EBITDA'].mean().reset_index()
    sector_stats.rename(columns={'EV/EBITDA': 'Sector_Avg_EV_EBITDA'}, inplace=True)
    sector_stats['Sector_Avg_EV_EBITDA'] = sector_stats['Sector_Avg_EV_EBITDA'].round(2)
    
    # Print the Benchmark Table
    print(sector_stats.to_string(index=False))
    
    # 5. FILTER: STOCK < SECTOR AVERAGE
    final_df = pd.merge(merged_df, sector_stats, on='Sector', how='left')
    
    # The Burry Filter: Value must be lower than the peer average
    burry_picks = final_df[final_df['EV/EBITDA'] < final_df['Sector_Avg_EV_EBITDA']].copy()
    
    # Calculate "Discount" metric for sorting
    burry_picks['Discount_%'] = round((1 - (burry_picks['EV/EBITDA'] / burry_picks['Sector_Avg_EV_EBITDA'])) * 100, 2)
    
    # Sort by the biggest discount relative to sector
    burry_picks = burry_picks.sort_values(by='Discount_%', ascending=False)
    
    return burry_picks

# ==========================================
# EXECUTION
# ==========================================

# Ensure we use the fortress_df from previous steps
if 'fortress_df' in locals() and not fortress_df.empty:
    
    Fortress_Burry_EV_EBITDA = filter_burry_ev_ebitda(fortress_df)
    
    if not Fortress_Burry_EV_EBITDA.empty:
        print(f"\n✅ Found {len(Fortress_Burry_EV_EBITDA)} Undervalued Stocks (Cheaper than Sector Avg).")
        print("Created variable: 'Fortress_Burry_EV_EBITDA'")
        
        # Display for Data Wrangler
        display(Fortress_Burry_EV_EBITDA[['Ticker', 'Sector', 'Price', 'EV/EBITDA', 'Sector_Avg_EV_EBITDA', 'Discount_%', 'Tier']])
    else:
        print("No stocks found trading below their sector average.")
else:
    print("⚠️ 'fortress_df' variable not found. Please run Step 1-3 first.")

📉 Analyzing EV/EBITDA for 267 Fortress stocks...

--- 📊 SECTOR AVERAGES (EV/EBITDA) ---
                Sector  Sector_Avg_EV_EBITDA
       Basic Materials                 11.38
Communication Services                 18.22
     Consumer Cyclical                 12.45
    Consumer Defensive                 16.04
                Energy                  7.18
            Healthcare                 16.59
           Industrials                 17.07
            Technology                 19.60
             Utilities                 19.46

✅ Found 160 Undervalued Stocks (Cheaper than Sector Avg).
Created variable: 'Fortress_Burry_EV_EBITDA'


,Ticker,Sector,Price,EV/EBITDA,Sector_Avg_EV_EBITDA,Discount_%,Tier
204,RDY,Healthcare,14.04,0.04,16.59,99.76,Fortress
218,HMY,Basic Materials,19.90,0.08,11.38,99.30,Fortress
254,PAGS,Technology,9.64,0.40,19.60,97.96,Fortress
231,GGB,Basic Materials,3.69,2.10,11.38,81.55,Fortress
160,PDD,Consumer Cyclical,113.39,2.38,12.45,80.88,Fortress
23,AVGO,Technology,346.10,4.90,19.60,75.00,Fortress
188,EQNR,Energy,23.63,1.85,7.18,74.23,Fortress
255,PLTK,Communication Services,3.95,5.60,18.22,69.26,Fortress
50,PLAB,Technology,32.00,6.06,19.60,69.08,Fortress
137,PPC,Consumer Defensive,38.99,5.10,16.04,68.20,Fortress


In [27]:
# ==========================================
# Watchlist Combiner (Finviz + YFinance)
# ==========================================



import pandas as pd
import yfinance as yf
from finvizfinance.quote import finvizfinance
import time
import numpy as np

# --- 1. INPUT YOUR MANUAL LIST HERE ---
MY_TICKERS = ['GRND','ARCC','BANC','ONB','UBER','ADMA','MIR','APG','SEI','FLEX','DD','SVM'] 

def get_combined_watchlist(ticker_list):
    print(f"--- Processing {len(ticker_list)} stocks ---")
    
    # --- PART A: Get Analyst Ratings from Finviz ---
    print("1. Fetching Analyst Ratings from Finviz...")
    finviz_data = []
    
    for ticker in ticker_list:
        try:
            stock = finvizfinance(ticker)
            info = stock.ticker_fundament()
            
            finviz_data.append({
                'Ticker': ticker,
                'Recom': info.get('Recom', np.nan),
                'Target_Price': info.get('Target Price', np.nan)
            })
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"   Skipping Finviz for {ticker}: {e}")
            finviz_data.append({'Ticker': ticker, 'Recom': np.nan, 'Target_Price': np.nan})

    df_finviz = pd.DataFrame(finviz_data)
    
    # --- PART B: Get Real-Time Stats from yfinance ---
    print("2. Fetching Price & Volatility from yfinance...")
    
    try:
        # Download data (1 Year is perfect for 52-Week MA)
        data = yf.download(ticker_list, period="1y", interval="1d", group_by='ticker', progress=False, threads=True)
        yf_stats = []
        
        for ticker in ticker_list:
            try:
                # --- FIXED: Robust Data Extraction ---
                if isinstance(data.columns, pd.MultiIndex):
                    if ticker in data.columns.levels[0]:
                        df = data[ticker].copy()
                    else:
                        print(f"   Warning: {ticker} not found in yfinance download.")
                        continue
                else:
                    df = data.copy()

                # Cleanup
                df = df.dropna(subset=['Close'])
                if len(df) < 20: 
                    print(f"   Warning: Not enough data for {ticker}")
                    continue

                # --- MATH CALCULATIONS ---
                current_price = df['Close'].iloc[-1]
                prev_close = df['Close'].iloc[-2]
                
                high_52 = df['High'].max()
                drop_from_high = ((current_price - high_52) / high_52) * 100
                
                change_pct = ((current_price - prev_close) / prev_close) * 100
                
                # Volatility (30-day Std Dev)
                volatility = df['Close'].pct_change().std() * 100
                
                # Relative Volume
                curr_vol = df['Volume'].iloc[-1]
                avg_vol = df['Volume'].tail(30).mean()
                rel_vol = curr_vol / avg_vol if avg_vol > 0 else 0

                # --- NEW: 52-Week Moving Average ---
                # Since we fetched exactly 1 year ('1y'), the mean of the whole column is the 52W MA
                ma_52w = df['Close'].mean()

                # Distance from MA (Optional but helpful metric)
                # dist_ma = ((current_price - ma_52w) / ma_52w) * 100 

                yf_stats.append({
                    'Ticker': ticker,
                    'Price': round(current_price, 2),
                    'Change_%': round(change_pct, 2),
                    '52W_MA': round(ma_52w, 2),          # <--- Added Here
                    'Drop_from_High_%': round(drop_from_high, 2),
                    'Volatility_%': round(volatility, 2),
                    'Rel_Volume': round(rel_vol, 2)
                })
                
            except Exception as e:
                print(f"   Error calculating stats for {ticker}: {e}")
                continue
                
        df_yf = pd.DataFrame(yf_stats)
        
    except Exception as e:
        print(f"yfinance Critical Error: {e}")
        return pd.DataFrame()

    # --- PART C: Merge ---
    if not df_finviz.empty:
        if not df_yf.empty:
            master_df = pd.merge(df_finviz, df_yf, on='Ticker', how='outer')
        else:
            master_df = df_finviz
            
        # Added '52W_MA' to this list so it displays in the final table
        cols = ['Ticker', 'Price', 'Change_%', '52W_MA', 'Drop_from_High_%', 'Recom', 'Target_Price', 'Rel_Volume', 'Volatility_%']
        
        final_cols = [c for c in cols if c in master_df.columns]
        return master_df[final_cols]
    else:
        return pd.DataFrame()

# --- RUN IT ---
watchlist_df = get_combined_watchlist(MY_TICKERS)

if not watchlist_df.empty:
    if 'Drop_from_High_%' in watchlist_df.columns:
        watchlist_df['Drop_from_High_%'] = pd.to_numeric(watchlist_df['Drop_from_High_%'], errors='coerce')
        print("\n--- Final Watchlist ---")
        display(watchlist_df.sort_values(by='Drop_from_High_%', ascending=True))
    else:
        display(watchlist_df)
else:
    print("No data found.")

--- Processing 12 stocks ---
1. Fetching Analyst Ratings from Finviz...
2. Fetching Price & Volatility from yfinance...

--- Final Watchlist ---


,Ticker,Price,Change_%,52W_MA,Drop_from_High_%,Recom,Target_Price,Rel_Volume,Volatility_%
6,GRND,13.54,0.97,17.62,-46.12,1.40,21.75,0.83,3.18
0,ADMA,18.24,-0.65,17.91,-28.94,1.00,30.00,0.76,3.26
7,MIR,23.42,-1.18,19.97,-22.65,1.12,30.62,0.71,3.49
11,UBER,81.71,-0.50,84.70,-19.88,1.47,112.40,0.42,2.39
9,SEI,45.97,-0.28,32.67,-19.41,1.17,65.45,0.69,5.90
5,FLEX,60.42,-2.03,48.48,-16.34,1.50,76.00,0.31,2.91
2,ARCC,20.23,-0.30,20.48,-9.64,1.27,22.64,0.89,1.38
10,SVM,8.34,-2.80,4.91,-9.05,1.17,9.43,0.57,3.62
8,ONB,22.31,-1.33,21.41,-6.56,1.85,25.92,0.76,2.14
1,APG,38.26,-1.54,31.52,-5.72,1.45,43.40,1.01,1.93


In [28]:
import requests
import pandas as pd
import google as genai
import enum
from typing_extensions import TypedDict
import json
import plotly.express as px
import sys
#!"{sys.executable}" -m pip install google.genai
#!"{sys.executable}" -m pip install plotly.express

In [29]:
tickers_gemini = ['SVM'] 
import os
from google import genai
from google.genai import types
from IPython.display import display, Markdown

In [30]:


# ==========================================
# SECURE CONFIGURATION
# ==========================================

# 1. Define the path to your key file
# If the file is in the same folder as this notebook, just use the filename.
KEY_FILE_PATH = "C:\\Users\\James\\OneDrive - McMaster University\\Gemini API Key\\gemini_key.txt"

def load_api_key(filepath):
    """
    Reads the API key from a local file to avoid hardcoding it.
    """
    try:
        with open(filepath, "r") as f:
            # .strip() removes any accidental newlines or spaces
            return f.read().strip()
    except FileNotFoundError:
        print(f"❌ Error: Could not find the file '{filepath}'")
        print("Please create a text file with your API key in it.")
        return None
    except Exception as e:
        print(f"❌ Error reading key file: {e}")
        return None

# 2. Load the key and set the environment variable
api_key = load_api_key(KEY_FILE_PATH)

if api_key:
    os.environ["GEMINI_API_KEY"] = api_key
    print("✅ API Key loaded securely.")
else:
    print("⚠️ CRITICAL: API Key not loaded. The script will fail.")

# ==========================================
# SENTIMENT ANALYSIS FUNCTION
# ==========================================
def analyze_sentiment_gemini_3(tickers_gemini, company_name=None):
    
    if not os.environ.get("GEMINI_API_KEY"):
        print("❌ Stop: No API Key found.")
        return

    print(f"\n🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing ${tickers_gemini}...")

    # Initialize Client
    client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

    config = types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(
            include_thoughts=False, 
            thinking_level="HIGH"
        ),
        tools=[types.Tool(
            google_search=types.GoogleSearch() 
        )],
        response_modalities=["TEXT"]
    )

    prompt = f"""
    You are a Senior Equity Research Analyst using the Gemini 3 Reasoning Engine. 
    Perform a deep "Market Sentiment Analysis" on {tickers_gemini} ({company_name if company_name else 'the company'}).
    
    Step 1: SEARCH. Use Google Search to find the latest (last 30 days) news, analyst notes, and SEC filings.
    Step 2: REASON. Analyze the search results to determine the true market psychology. Look for contradictions between price action and news.
    
    Investigate these 4 Pillars:
    1. **News Virality**: Are headlines fear-mongering or euphoric? (Look for scandals, lawsuits, or product breakthroughs).
    2. **Analyst Shifts**: Are price targets moving UP or DOWN in the last week?
    3. **Institutional Flows**: Any reports of hedge funds or insiders buying/selling?
    4. **The "Whisper" Number**: What are traders saying on forums vs. official guidance?

    **OUTPUT FORMAT:**
    Produce a professional Markdown report:
    
    ## 🧠 Gemini 3 Sentiment Report: {tickers_gemini}
    **Reasoning Depth:** High
    **Sentiment Score:** [1-10]
    **Verdict:** [Buy / Hold / Sell / Speculative]
    
    ### 1. The Bull Thesis (Why it goes up)
    * ...
    
    ### 2. The Bear Thesis (Why it goes down)
    * ...
    
    ### 3. Deep Dive Analysis
    * **News Analysis**: ...
    * **Smart Money**: ...
    * **Financial Statement Analysis**: (Historic performance over last 3 years + expected performance)
    
    ### 4. Conclusion
    [Summary of whether the current price is a trap or an opportunity]
    """

    try:
        response = client.models.generate_content(
            model='gemini-3-flash-preview', # Or 'gemini-3-flash-preview'
            contents=prompt,
            config=config
        )
        display(Markdown(response.text))
        
    except Exception as e:
        print(f"❌ Error: {e}")

# ==========================================
# EXECUTION
# ==========================================
# Only run this if the key loaded successfully
if os.environ.get("GEMINI_API_KEY"):
    analyze_sentiment_gemini_3(tickers_gemini, tickers_gemini)

✅ API Key loaded securely.

🧠 Gemini 3 is thinking (High Reasoning Mode)... analyzing $['SVM']...


## 🧠 Gemini 3 Sentiment Report: Silvercorp Metals Inc. (SVM)
**Reasoning Depth:** High  
**Sentiment Score:** 7.8/10  
**Verdict:** Speculative Buy / Growth Opportunity

### 1. The Bull Thesis (Why it goes up)
*   **Operational Tailwinds & Silver Price Surge**: SVM is riding a macro wave as spot silver prices approached $75-$80 levels in late 2025. The company reported record-breaking Fiscal 2025 results (ending March 31, 2025) with revenue up 39% and net income up 60%.
*   **Expansion & Diversification**: Growth is not just dependent on silver prices; the company is aggressively expanding its mill capacity (Ying District) and progressing on the El Domo project in Ecuador, signaling a transition toward becoming a more diversified base-metal producer (Copper/Gold).
*   **Earnings Momentum**: SVM has consistently beaten EPS estimates in recent quarters (e.g., Q1 and Q2 Fiscal 2026 both posted $0.10 vs. $0.09 estimates).
*   **Strong Technical Setup**: Despite year-end volatility, the stock is trading near 52-week highs ($8.90+) and maintains a strong rising trend, supported by its 50-day and 200-day moving averages.

### 2. The Bear Thesis (Why it goes down)
*   **Insider "Exit" Signals**: A significant red flag is the clustering of insider sales. In December 2025, Director Yikang Liu sold 8.11% of his holding, following earlier sales by the CEO/Chairman Rui Feng ($1.7M) and the CFO. This suggests management may believe the stock is nearing a local peak.
*   **Geopolitical & Concentration Risk**: The majority of production remains in China. While SVM has a long history of navigating Chinese regulations, this concentration remains a systemic risk for North American investors.
*   **Net Income Volatility**: While *adjusted* net income is strong, the unadjusted figures have shown volatility due to non-cash charges ($53M charge on derivative liabilities in Q2 2026), which can spook retail investors who focus on headline numbers.

### 3. Deep Dive Analysis

*   **News Analysis (Virality: Neutral-Euphoric)**: Headlines are dominated by "52-Week Highs" and "Record Financials." There is very little "fear" in the news cycle, though some technical analysis platforms note a high "Fear & Greed Index" (39) for the broader sector, suggesting the silver trade is becoming "crowded."
*   **Smart Money (Institutional Flow)**: 
    *   **Institutions**: Institutional ownership is healthy (~46%). The Amplify Junior Silver Miners ETF (SILJ) recently reported a passive stake of 5.01%.
    *   **Insiders**: The narrative here is a "Sell." With multiple executives offloading shares as the stock hit the $8.00–$9.00 range, "Smart Money" at the corporate level is taking profits.
*   **Financial Statement Analysis (3-Year Trend)**:
    *   **2023**: Revenue $208M | Net Income $21M (Stable base).
    *   **2024**: Revenue $215M | Net Income $36M (Beginning of the silver rally).
    *   **2025**: Revenue $299M | Net Income $58M (Record year driven by price + volume).
    *   **2026 Forecast**: Analysts expect EPS to jump to $0.54 for the full year, a significant leap from prior levels, fueled by higher realized metal prices.
*   **The "Whisper" Number**: 
    *   **Official Consensus**: Analysts are looking for $0.15 EPS for the upcoming Q3 2026 report (Feb 10, 2026).
    *   **Trader Sentiment**: On forums like Stocktwits and Reddit, traders are whispering a beat closer to **$0.17 - $0.18**, citing higher silver/gold selling prices realized during the holiday quarter and the reopening of previously closed mining areas in the Ying District.

### 4. Conclusion: Trap or Opportunity?
**Opportunity (with caution).** 
The current price of $8.90 is not a "trap" in the sense of a failing business, but it is a "crowded trade." The fundamental growth is undeniable—SVM is a rare mining company that combines high cash flow ($138M CFO) with a massive cash pile ($382M) and a dividend. 

**Strategy**: The insider selling suggests a pullback to the **$7.80 - $8.20** support level is likely before the next leg up. Long-term investors should view any "insider-led" dips as a buying opportunity, whereas short-term traders should wait for the "Whisper" to be confirmed in February before chasing the current 52-week high.